# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0           taro  -8.3862  115.2800     80.60        48          49   
1        1  blackmans bay -43.0167  147.3167     61.74        60          70   
2        2        udachny  66.4167  112.4000    -11.07        85          93   
3        3      adamstown -25.0660 -130.1015     73.15        81         100   
4        4  atafu village  -8.5421 -172.5159     85.24        74          95   

   Wind Speed Country        Date  
0        5.50      ID  1703052565  
1        1.99      AU  1703052565  
2        5.12      RU  1703052565  
3       15.12      PN  1703052565  
4       10.31      TK  1703052565

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity", 
    color = "City"
)


# Display the map
city_humidity

C:\Users\School\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal weather is between 50-70 degrees fahrenheit
# with humidity between 40-60 and cloudiness less than 50
my_ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 50)
                                    & (city_data_df["Max Temp"] < 70)
                                    & (city_data_df["Humidity"] > 40)
                                    & (city_data_df["Humidity"] < 60)
                                    & (city_data_df["Cloudiness"] < 50)]


# Drop any rows with null values
my_ideal_weather.dropna()

# Display sample data
my_ideal_weather

City_ID                City      Lat       Lng  Max Temp  Humidity  \
7          7            luderitz -26.6481   15.1594     67.95        52   
14        14        port lincoln -34.7333  135.8667     63.32        55   
122      122       mount gambier -37.8333  140.7667     61.52        50   
147      147          codrington -38.2667  141.9667     61.66        51   
194      194             al jawf  29.5000   38.7500     50.31        57   
241      241  san pedro pochutla  15.7333  -96.4667     67.19        45   
305      305    diego de almagro -26.3667  -70.0500     61.63        56   
368      368               smara  26.7384  -11.6719     50.68        50   
460      460           masterton -40.9597  175.6575     65.88        56   
519      519            tubarjal  30.4999   38.2160     50.18        49   

     Cloudiness  Wind Speed Country        Date  
7             0       14.03     NaN  1703052565  
14           18       16.49      AU  1703052566  
122           3       14.14      AU  1703052577  
147          39       18.48      AU  1703052579  
194           1        8.68      SA  1703052583  
241           0        3.96      MX  1703052586  
305           0        1.50      CL  1703052592  
368           0        4.83      EH  1703052597  
460           1        4.85      NZ  1703052610  
519           0        9.08      SA  1703052617

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
7              luderitz     NaN -26.6481   15.1594        52           
14         port lincoln      AU -34.7333  135.8667        55           
122       mount gambier      AU -37.8333  140.7667        50           
147          codrington      AU -38.2667  141.9667        51           
194             al jawf      SA  29.5000   38.7500        57           
241  san pedro pochutla      MX  15.7333  -96.4667        45           
305    diego de almagro      CL -26.3667  -70.0500        56           
368               smara      EH  26.7384  -11.6719        50           
460           masterton      NZ -40.9597  175.6575        56           
519            tubarjal      SA  30.4999   38.2160        49

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", 
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary & Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
port lincoln - nearest hotel: Boston Hotel
mount gambier - nearest hotel: The Old Mount Gambier Gaol
codrington - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
san pedro pochutla - nearest hotel: Olimpia
diego de almagro - nearest hotel: Residencial Hambyll
smara - nearest hotel: Hôtel Amine فندق الأمين
masterton - nearest hotel: Copthorne Hotel Solway Park
tubarjal - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
7              luderitz     NaN -26.6481   15.1594        52   
14         port lincoln      AU -34.7333  135.8667        55   
122       mount gambier      AU -37.8333  140.7667        50   
147          codrington      AU -38.2667  141.9667        51   
194             al jawf      SA  29.5000   38.7500        57   
241  san pedro pochutla      MX  15.7333  -96.4667        45   
305    diego de almagro      CL -26.3667  -70.0500        56   
368               smara      EH  26.7384  -11.6719        50   
460           masterton      NZ -40.9597  175.6575        56   
519            tubarjal      SA  30.4999   38.2160        49   

                        Hotel Name  
7    Seaview Hotel Zum Sperrgebiet  
14                    Boston Hotel  
122     The Old Mount Gambier Gaol  
147                 No hotel found  
194                 No hotel found  
241                        Olimpia  
305            Residencial Hambyll  
368        Hôtel Amine فندق الأمين  
460    Copthorne Hotel Solway Park  
519                 No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
#    tiles = "EsriNatGeo", 
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)